In [17]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_blobs
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import GridSearchCV

In [10]:
path="/home/sultan/EEG/Source_level_frequency_analysis/100sam_NHIerp_band_features_clear.csv"
# path="/home/sultan/EEG/Source_Level_Analysis/125sam_10ms_clear_all_erp.csv"
Metadata=pd.read_csv(path)
Metadata.head(2)
X=Metadata.iloc[:,2:]
y=Metadata.iloc[:,1]
X.shape,y.shape

# X=preprocessing.scale(X)
scaler = preprocessing.StandardScaler()
X = scaler.fit_transform(X)
print X.shape,y.shape
from sklearn.preprocessing import label_binarize
y=label_binarize(np.asarray(y),[0,1])

(1861, 272) (1861,)


In [11]:
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.cross_validation import train_test_split

# datax,datay=make_blobs(n_samples=2000,random_state=1)
trainx, testx, trainy, testy = train_test_split(X,y,test_size=0.20, random_state=1)


###############SVC with probability=True
clf=SVC(probability=True,random_state=1)
clf.fit(trainx,trainy)
y_pred=clf.predict(testx)
print 'SVC ',accuracy_score(testy,y_pred)

###############AdaBoostClassifier with SAMME.R
clf=AdaBoostClassifier(algorithm='SAMME.R',base_estimator=SVC(probability=True,random_state=1),random_state=1,n_estimators=50)
clf.fit(trainx,trainy)
y_pred=clf.predict(testx)
print 'AdaBoostClassifier',accuracy_score(testy,y_pred)



###############SVC with probability=False
clf=SVC(random_state=1)
clf.fit(trainx,trainy)
y_pred=clf.predict(testx)
print 'SVC ',accuracy_score(testy,y_pred)
###############AdaBoostClassifier with SAMM
clf=AdaBoostClassifier(algorithm='SAMME',base_estimator=SVC(random_state=1),random_state=1,n_estimators=10)
clf.fit(trainx,trainy)
y_pred=clf.predict(testx)
print 'AdaBoostClassifier',accuracy_score(testy,y_pred)

/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC  0.9195710455764075
AdaBoostClassifier 0.6702412868632708
SVC  0.9195710455764075
AdaBoostClassifier 0.5924932975871313


In [15]:
from sklearn.model_selection import KFold
from sklearn import tree
from sklearn.model_selection import cross_val_score
crossvalidation=KFold(n_splits=10,shuffle=True,random_state=1)
for depth in range (1,10):
    tree_classifier=tree.DecisionTreeClassifier(max_depth=depth,random_state=1)
    if tree_classifier.fit(X,y).tree_.max_depth<depth:
        break
    score=np.mean(cross_val_score(tree_classifier,X,y,scoring='accuracy', cv=crossvalidation,n_jobs=1))
    print(depth, score)

(1, 0.7748605600598011)
(2, 0.7721752630671037)
(3, 0.7829107009372664)
(4, 0.8124633431085044)
(5, 0.8178339370938991)
(6, 0.8232045310792939)
(7, 0.8269593467885687)
(8, 0.8274998562474843)
(9, 0.8178425622448392)


In [18]:
ada=AdaBoostClassifier()
search_grid={'n_estimators':[500,1000,2000],'learning_rate':[.001,0.01,.1]}
search=GridSearchCV(estimator=ada,param_grid=search_grid,scoring='accuracy',n_jobs=1,cv=crossvalidation)

In [20]:
search.fit(X,y)
search.best_params_
search.best_score_

0.92423428264374

In [ ]:
# C_range = np.logspace(-2, 2, 5)
# # gamma_range = np.logspace(-2, 2, 5)
# gamma_range = [0.01,0.002,0.00069,0.0007,0.0005]
# param_grid = dict(gamma=gamma_range, C=C_range)

In [ ]:
# ada=AdaBoostClassifier()
# search_grid={'n_estimators':[500,1000,2000],'learning_rate':[.001,0.01,.1]}
# search=GridSearchCV(estimator=ada,param_grid=search_grid,scoring='accuracy',n_jobs=1,cv=crossvalidation)